In [1]:
import arcpy
import logging
from arcpy import env
import os

from src.utils import arcpy_utils as au

# set up logger
setup_logging()

In [2]:
def voronoi(polygon_layer, point_layer, filegdb_path, v_crowns_c2_split, area_extent):
    """_summary_

    Parameters
    ----------
    polygon_layer : _type_
        _description_
    point_layer : _type_
        _description_
    filegdb_path : _type_
        _description_
    v_crowns_c2_split : _type_
        _description_
    """
    logger = logging.getLogger(__name__)
    count_crowns = int(arcpy.GetCount_management(polygon_layer).getOutput(0))
    logger.info(f"Count of Case 2 Crowns: {count_crowns}")

    # Split each tree crown based on the number of stems.
    fields = ["OBJECTID", "crown_id"]
    with arcpy.da.SearchCursor(polygon_layer, fields) as cursor:
        for row in cursor:
            logger.info(
                f"START SPLITTING TREECROWN, OBJECTID: {row[0]}, crown_id: {row[1]}"
            )
            # reset environment extent!
            env.extent = polygon_layer
            # select tree crown by crown_id
            polygon_id = str(row[1])

            # temporary layers that have to be deleted and cleared!
            tmp_crown_lyr = os.path.join(filegdb_path, "temp_crown_" + polygon_id)
            tmp_thiessen_lyr = os.path.join(filegdb_path, "temp_thiessen_" + polygon_id)
            tmp_split_crown_lyr = os.path.join(
                filegdb_path, "temp_split_crown_" + polygon_id
            )
            tmp_selected_points = os.path.join(
                filegdb_path, "temp_selected_points_" + polygon_id
            )

            # create a layer for the selected polygon
            arcpy.MakeFeatureLayer_management(
                polygon_layer, "selected_polygon", f"CROWN_ID = '{polygon_id}'"
            )
            arcpy.CopyFeatures_management(
                "selected_polygon", tmp_crown_lyr
            )  # tree crown

            # Create point layer
            arcpy.MakeFeatureLayer_management(point_layer, "point_lyr")
            arcpy.SelectLayerByLocation_management(
                in_layer="point_lyr",  # selected points
                overlap_type="INTERSECT",
                select_features=tmp_crown_lyr,  # crown
                selection_type="NEW_SELECTION",
            )

            # selected points
            arcpy.CopyFeatures_management(
                "point_lyr", tmp_selected_points
            )  # tree crown

            # log the tree_id values of the selected stem points
            fields_pnt = ["OBJECTID", "tree_id"]
            with arcpy.da.SearchCursor(tmp_selected_points, fields_pnt) as cursor:
                for row in cursor:
                    logger.info(f"selected_point: {row[0]}, tree_id: {row[1]}")

                # split the treecrown using the thiessen polygons
            env.extent = tmp_crown_lyr  # tree crown area
            env.overwriteOutput = True

            arcpy.CreateThiessenPolygons_analysis(tmp_selected_points, tmp_thiessen_lyr)
            arcpy.Intersect_analysis(
                [tmp_thiessen_lyr, tmp_crown_lyr], tmp_split_crown_lyr
            )

            # Create featureclass if not exists
            if not arcpy.Exists(v_crowns_c2_split):
                arcpy.CreateFeatureclass_management(
                    filegdb_path,
                    "crowns_c2_split",
                    geometry_type="POLYGON",
                    spatial_reference=tmp_split_crown_lyr,
                )
                # add fields to the new feature class
                field_list = [
                    "geo_relation",
                    "stem_count",
                    "seg_method",
                    "bydelnummer",
                    "crown_id",
                    "tree_height_laser",
                    "tree_altit",
                ]
                field_type = [
                    "TEXT",
                    "LONG",
                    "TEXT",
                    "TEXT",
                    "TEXT",
                    "FLOAT",
                    "FLOAT",
                ]

                for field, field_type in zip(field_list, field_type):
                    au.addField_ifNotExists(v_crowns_c2_split, field, field_type)

                logger.info(
                    "Target feature class '{}' created.".format(v_crowns_c2_split)
                )

            # append the splitted crowns to crowns_c2_split
            arcpy.Append_management(
                inputs=tmp_split_crown_lyr,
                target=v_crowns_c2_split,
                schema_type="NO_TEST",
                field_mapping=None,
                subtype="",
                expression="",
                match_fields=None,
                update_geometry="NOT_UPDATE_GEOMETRY",
            )

            count_appended_crowns = int(
                arcpy.GetCount_management(tmp_split_crown_lyr).getOutput(0)
            )
            logger.info(f"Appended crowns: {count_appended_crowns}")

            # uncomment for complet logging
            # fields_polygon= ["OBJECTID", "crown_id"]
            # with arcpy.da.SearchCursor(v_crowns_c2_split, fields_polygon) as cursor:
            #     for row in cursor:
            #         logger.info(
            #             f"Appended crowns: {row[0]},  crown_id: {row[1]}"
            #         )

            logger.info(
                "Clear selection and delete temporary layers, BEFORE moving to the next crown.."
            )
            lyr_list = [
                "point_lyr",
                "selected_polygon",
                tmp_crown_lyr,
                tmp_thiessen_lyr,
                tmp_split_crown_lyr,
                tmp_selected_points,
            ]

            lyr_list = [lyr for lyr in lyr_list if au.exists_and_has_features(lyr)]

            # clear and delete
            for lyr in lyr_list:
                arcpy.SelectLayerByAttribute_management(lyr, "CLEAR_SELECTION")
                arcpy.Delete_management(lyr)

            # solely clear selection (DO NOT DELETE)
            if au.exists_and_has_features(point_layer):
                arcpy.SelectLayerByAttribute_management(point_layer, "CLEAR_SELECTION")
            if au.exists_and_has_features(polygon_layer):
                arcpy.SelectLayerByAttribute_management(
                    polygon_layer, "CLEAR_SELECTION"
                )

    # reset extent
    env.extent = area_extent

In [3]:
polygon_layer = r"P:\152022_itree_eco_ifront_synliggjore_trars_rolle_i_okosyst\data\oslo\urban-tree-detection\oslo_geo_relation.gdb\crowns_C3_bym_2018"
point_layer = r"P:\152022_itree_eco_ifront_synliggjore_trars_rolle_i_okosyst\data\oslo\general\oslo_traer\oslo_bytraer.gdb\BYM_registrerte_traer_2018"
filegdb_path = r"P:\152022_itree_eco_ifront_synliggjore_trars_rolle_i_okosyst\data\oslo\urban-tree-detection\oslo_geo_relation.gdb"
v_crowns_c2_split = r"P:\152022_itree_eco_ifront_synliggjore_trars_rolle_i_okosyst\data\oslo\urban-tree-detection\oslo_geo_relation.gdb\crowns_c2_split"
area_extent = r"P:\152022_itree_eco_ifront_synliggjore_trars_rolle_i_okosyst\data\oslo\general\oslo_admindata.gdb\kommune"

voronoi(polygon_layer, point_layer, filegdb_path, v_crowns_c2_split, area_extent)
